# Lab Assignment #6 – Using Maps and Hash Tables

## Exercise 1

**If your first name starts with a letter from A-J inclusively:**

Our `AbstractHashMap` class maintains a load factor l ≤ 0.5. Reimplement
that class to allow the user to specify the maximum load, and adjust the
concrete subclasses accordingly.

Perform experiments on our `ProbeHashMap` classes to measure its
efficiency using random key sets and varying limits on the load factor.
Do you think `ProbeHashMap` is better or `ChainHashMap`? When and how?

**Hint** The load factor can be controlled from within the abstract
class, but there must be means for setting the parameter (either through
the constructor, or a new method).

Write a Java/Python application to test your solution.

In [4]:
import time
from random import randrange

from probe_hash_map import ProbeHashMap


def test_efficiency(max_load_factors, num_items):
    results = {}
    for max_load in max_load_factors:
        hashmap = ProbeHashMap(max_load=max_load)
        start_time = time.time()

        # Insert random key-value pairs
        for _ in range(num_items):
            key = randrange(10 * num_items)
            value = randrange(10 * num_items)
            hashmap[key] = value

        # Measure insertion time
        insert_time = time.time() - start_time

        # Measure retrieval time
        start_time = time.time()
        for _ in range(num_items):
            key = randrange(10 * num_items)
            try:
                _ = hashmap[key]
            except KeyError:
                pass
        retrieval_time = time.time() - start_time

        results[max_load] = (insert_time, retrieval_time)
    return results


def main():
    max_load_factors = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    num_items = 1000
    results = test_efficiency(max_load_factors, num_items)

    print("Load Factor | Insert Time (s) | Retrieval Time (s)")
    print("-----------------------------------------------")
    for max_load, (insert_time, retrieval_time) in results.items():
        print(f"{max_load:.2f}       | {insert_time:.6f}      | {retrieval_time:.6f}")


if __name__ == "__main__":
    main()

Load Factor | Insert Time (s) | Retrieval Time (s)
-----------------------------------------------
0.10       | 0.013463      | 0.002127
0.20       | 0.007283      | 0.002745
0.30       | 0.007456      | 0.001964
0.40       | 0.005632      | 0.002085
0.50       | 0.006297      | 0.004177
0.60       | 0.009743      | 0.002105
0.70       | 0.016293      | 0.003860
0.80       | 0.006396      | 0.003594
0.90       | 0.007616      | 0.003833


**If your first name starts with a letter from K-Z inclusively:**

Our `AbstractHashMap` class maintains a load factor l ≤ 0.5. Reimplement
that class to allow the user to specify the maximum load, and adjust the
concrete subclasses accordingly.

Perform experiments on our `ChainHashMap` classes to measure its
efficiency using random key sets and varying limits on the load factor.
Do you think `ProbeHashMap` is better or `ChainHashMap`? When and how?

**Hint** The load factor can be controlled from within the abstract
class, but there must be means for setting the parameter (either through
the constructor, or a new method).

Write a Java/Python application to test your solution

In [5]:
import time
from random import randint

from chain_hash_map import ChainHashMap


def test_efficiency(max_load_factors, num_items):
    results = {}
    for max_load in max_load_factors:
        hashmap = ChainHashMap(max_load=max_load)
        start_time = time.time()

        # Insert random key-value pairs
        for _ in range(num_items):
            key = randint(0, 10 * num_items)
            value = randint(0, 10 * num_items)
            hashmap[key] = value

        # Measure insertion time
        insert_time = time.time() - start_time

        # Measure retrieval time
        start_time = time.time()
        for _ in range(num_items):
            key = randint(0, 10 * num_items)
            try:
                _ = hashmap[key]
            except KeyError:
                pass
        retrieval_time = time.time() - start_time

        results[max_load] = (insert_time, retrieval_time)
    return results


def main():
    max_load_factors = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    num_items = 1000
    results = test_efficiency(max_load_factors, num_items)

    print("Load Factor | Insert Time (s) | Retrieval Time (s)")
    print("-----------------------------------------------")
    for max_load, (insert_time, retrieval_time) in results.items():
        print(f"{max_load:.2f}       | {insert_time:.6f}      | {retrieval_time:.6f}")


if __name__ == "__main__":
    main()

Load Factor | Insert Time (s) | Retrieval Time (s)
-----------------------------------------------
0.10       | 0.007558      | 0.001886
0.20       | 0.009452      | 0.001878
0.30       | 0.010275      | 0.003392
0.40       | 0.007042      | 0.001985
0.50       | 0.010714      | 0.003715
0.60       | 0.008236      | 0.001914
0.70       | 0.032945      | 0.001930
0.80       | 0.007297      | 0.002882
0.90       | 0.006703      | 0.001991


- We use **ProbeHashMap** when we expect fewer collisions and want better performance with a lower load factor (≤ 0.5).
- We use **ChainHashMap** when we need consistent performance even with a higher load factor (> 0.5).